<a href="https://colab.research.google.com/github/ranaehelal/The-Zebra-Puzzle/blob/main/Zebra_Puzzle_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install -r requirements.txt


In [44]:
!pip install ipythonblocks

In [45]:
!pip install sortedcontainers

In [46]:
from aima.logic import *
from aima.utils import expr
from aima.csp import *
import sys


In [47]:
# List of persons, potters, dreams, and desserts
persons = ["Peter","AuntPolly", "Betty", "Daddy", "Mummy"]
potters = ["Peter", "Betty", "Daddy", "Mummy"]
dreams = ["TripSea", "Ticket", "TripParis", "CoinCollection"]
desserts = ["Marmalade", "Marshmallows", "Waffles", "IceCream", "NapoleonCake"]
holiday_desserts = ["NapoleonCake", "Marmalade", "Waffles"]

# Initialize the clauses list
clauses = []

# Add base facts about hobbies
hobbies = {
    "Mummy": "Yoga",
    "AuntPolly": "Sewing",
    "Betty": "Ballet",
    "Daddy": "Fishing",
    "Peter": "CoinCollection"
}

for person, hobby in hobbies.items():
    clauses.append(expr(f"Hobby({person}, {hobby})"))

# Known dreams - explicit assignments
clauses.append(expr("Dream(Peter, CoinCollection)"))
clauses.append(expr("Dream(Betty, Ticket)"))


# Known desserts
clauses.append(expr("Desert(Mummy, Marshmallows)"))
clauses.append(expr("Desert(Betty, Marmalade)"))

# Add Potter status using a loop
for person in potters:
    clauses.append(expr(f"Potter({person})"))

# Add Person status using a loop
for person in persons:
    clauses.append(expr(f"Person({person})"))

# Core rules
clauses.append(expr("Person(x) & Desert(x, IceCream) ==> Dream(x, TripParis)"))

# Relationship between Daddy and Mummy's dreams
clauses.append(expr("Dream(Daddy, x) ==> Dream(Mummy, x)"))
clauses.append(expr("Dream(Mummy, x) & PotterDream(x) ==> Dream(Daddy, x)"))

# Known PotterDream
clauses.append(expr("PotterDream(TripSea)"))

# Facts about desserts containing cream
clauses.append(expr("ContainsCream(IceCream)"))
clauses.append(expr("ContainsCream(NapoleonCake)"))

clauses.append(expr("ContainsCream(y) ==> Desert_NotLike(Peter, y)"))

# Assigned desserts
assigned_desserts = {
    "Mummy": "Marshmallows",
    "Betty": "Marmalade",
}

for person, dessert in assigned_desserts.items():
    clauses.append(expr(f"HasAssignedDesert({person}, {dessert})"))

# Build the Knowledge Base using FOL (First-Order Logic)
kb = FolKB(clauses)




In [48]:
# Second query: Napoleon cake preferences
print("\nWho likes Napoleon cake?")

for person in potters:
    potter_query = expr(f"Potter({person})")
    assigned_query = expr(f"HasAssignedDesert({person}, x)")

    is_potter = kb.ask(potter_query)
    has_other_desert = kb.ask(assigned_query)

    if has_other_desert :
        dessert_query = expr(f"Desert({person}, x)")
        dessert = kb.ask(dessert_query)
        print(f"- {person} has already been assigned {dessert}")

    else:
        napoleon_dislike_query = expr(f"Desert_NotLike({person}, NapoleonCake)")
        dislikes_napoleon = kb.ask(napoleon_dislike_query)


        if not dislikes_napoleon:
            # If they don't dislike Napoleon cake, they like it
            print(f"- {person} likes Napoleon cake")
            # Add this preference to the knowledge base
            kb.tell(expr(f"Desert({person}, NapoleonCake)"))
        else:
            print(f"- {person} dislikes Napoleon cake")
            kb.tell(expr(f"Desert({person}, Waffles)"))





Who likes Napoleon cake?
- Peter dislikes Napoleon cake
- Betty has already been assigned {x: Marmalade}
- Daddy likes Napoleon cake
- Mummy has already been assigned {x: Marshmallows}


In [49]:
# Second query: Napoleon cake preferences
print("\nWho likes Napoleon cake?")

result_dislike = kb.ask(expr("Desert(x, NapoleonCake)"))

print(f" Result: {result_dislike}")




Who likes Napoleon cake?
 Result: {x: Daddy}


In [51]:
for dessert in desserts:
    dessert_liked_by_anyone = False

    print(f"\nChecking if anyone likes {dessert}...")

    # Check if anyone likes the dessert
    for person in persons:
        likes_dessert_query = expr(f"Desert({person}, {dessert})")
        likes_dessert = kb.ask(likes_dessert_query)

        if likes_dessert is not False:  # If it's not False, the query is satisfied
            dessert_liked_by_anyone = True
            print(f"Query: Does {person} like {dessert}? Result: {likes_dessert}")
            print(f"{person} likes {dessert}.")
            break  # Exit if someone likes this dessert
        else:
            print(f"Query: Does {person} like {dessert}? Result: {likes_dessert}")

    # If no one likes the dessert
    if not dessert_liked_by_anyone:
        print(f"No one likes {dessert}.")

        for person in persons:
            dislikes_all_desserts = True

            for other_dessert in desserts:
                likes_any_query = expr(f"Desert({person}, {other_dessert})")
                likes_any = kb.ask(likes_any_query)

                if likes_any is not False:  # If it's not False, the query is satisfied
                    print(f"Query: Does {person} like {other_dessert}? Result: {likes_any}")
                    dislikes_all_desserts = False
                    break  # Person likes at least one dessert
                else:
                    print(f"Query: Does {person} like {other_dessert}? Result: {likes_any}")

            print(f"Does {person} dislike all desserts? Result: {dislikes_all_desserts}")

            if dislikes_all_desserts:
                print(f"{person} dislikes all desserts. Assigning {dessert} to {person}.")
                kb.tell(expr(f"Desert({person}, {dessert})"))  # Add the fact to the KB
                break



Checking if anyone likes Marmalade...
Query: Does Peter like Marmalade? Result: False
Query: Does AuntPolly like Marmalade? Result: False
Query: Does Betty like Marmalade? Result: {}
Betty likes Marmalade.

Checking if anyone likes Marshmallows...
Query: Does Peter like Marshmallows? Result: False
Query: Does AuntPolly like Marshmallows? Result: False
Query: Does Betty like Marshmallows? Result: False
Query: Does Daddy like Marshmallows? Result: False
Query: Does Mummy like Marshmallows? Result: {}
Mummy likes Marshmallows.

Checking if anyone likes Waffles...
Query: Does Peter like Waffles? Result: {}
Peter likes Waffles.

Checking if anyone likes IceCream...
Query: Does Peter like IceCream? Result: False
Query: Does AuntPolly like IceCream? Result: False
Query: Does Betty like IceCream? Result: False
Query: Does Daddy like IceCream? Result: False
Query: Does Mummy like IceCream? Result: False
No one likes IceCream.
Query: Does Peter like Marmalade? Result: False
Query: Does Peter li

In [52]:
print("\nWho dreams of Paris?")
for person in people:
    # First check if they have an assigned dream
    has_assigned = fol_fc_ask(kb, expr(f"HasAssignedDream({person}, x)"))

    if not len(list(has_assigned)):  # Only check for Paris dreams if no assigned dream
        # Check ice cream rule
        Trip = fol_fc_ask(kb, expr(f"Dream({person}, TripParis)"))

        if  len(list(Trip)):
            print(f"- {person} dreams of TripParis (because they like ice cream)")
            kb.tell(expr(f"Dream({person}, TripParis)"))
    else:
        print(f"- {person} has an assigned dream and cannot dream of Paris")


Who dreams of Paris?
- AuntPolly dreams of TripParis (because they like ice cream)


In [53]:
persons = ["AuntPolly", "Peter", "Betty","Daddy","Mummy"]
potters = [ "Peter", "Betty","Daddy","Mummy"]
dreams = ["TripSea", "Ticket", "TripParis", "CoinCollection"]
desserts = ["Marmalade", "Marshmallows", "Waffles", "IceCream","NapoleonCake"]
holiday_desserts = ["NapoleonCake", "Marmalade", "Waffles"]


definite_clauses_KB = PropDefiniteKB()

# Person status
definite_clauses_KB.tell(expr("Person_Betty"))
definite_clauses_KB.tell(expr("Person_Peter"))
definite_clauses_KB.tell(expr("Person_Daddy"))
definite_clauses_KB.tell(expr("Person_Mummy"))
definite_clauses_KB.tell(expr("Person_AuntPolly"))

# Potter status
definite_clauses_KB.tell(expr("Potter_Mummy"))
definite_clauses_KB.tell(expr("Potter_Daddy"))
definite_clauses_KB.tell(expr("Potter_Peter"))
definite_clauses_KB.tell(expr("Potter_Betty"))

# deserts
definite_clauses_KB.tell(expr("Desert_Mummy_Marshmallows"))
definite_clauses_KB.tell(expr("Desert_Betty_Marmalade"))



# facts about hobbies
definite_clauses_KB.tell(expr("Hobby_Mummy_Yoga"))
definite_clauses_KB.tell(expr("Hobby_AuntPolly_Sewing"))
definite_clauses_KB.tell(expr("Hobby_Betty_Ballet"))
definite_clauses_KB.tell(expr("Hobby_Daddy_Fishing"))
definite_clauses_KB.tell(expr("Hobby_Peter_CoinCollection"))

# dreams
definite_clauses_KB.tell(expr("Dream_Peter_CoinCollection"))
definite_clauses_KB.tell(expr("Dream_Betty_Ticket"))


# Core rules

for person in persons:
        definite_clauses_KB.tell(expr(f"(Person_{person} & Desert_{person}_IceCream) ==> Dream_{person}_TripParis"))

for dream in dreams:
    definite_clauses_KB.tell(expr(f"Dream_Daddy_{dream} ==> Dream_Mummy_{dream}"))
    definite_clauses_KB.tell(expr(f"(Dream_Mummy_{dream} & PotterDream_{dream}) ==> Dream_Daddy_{dream}"))


definite_clauses_KB.tell(expr("PotterDream_TripSea"))


definite_clauses_KB.tell(expr("HasAssignedDesert_Mummy_Marshmallows"))
definite_clauses_KB.tell(expr("HasAssignedDesert_Betty_Marmalade"))


definite_clauses_KB.tell(expr("IceCream_ContainsCream"))
definite_clauses_KB.tell(expr("NapoleonCake_ContainsCream"))
for dessert in desserts:
        definite_clauses_KB.tell(expr(f"{dessert}_ContainsCream ==> Desert_Not_like_Peter_{dessert}"))





In [54]:
for person in potters:
    for dessert in holiday_desserts:
        not_assigned_any_dessert = True
        for other_dessert in desserts:
            assigned_dessert_query = expr(f"HasAssignedDesert_{person}_{other_dessert}")
            assigned = pl_fc_entails(definite_clauses_KB, assigned_dessert_query)

            not_assigned_any_dessert = not_assigned_any_dessert and (not assigned)

        dislike_query = expr(f"Desert_Not_like_{person}_{dessert}")
        dislikes_dessert = pl_fc_entails(definite_clauses_KB, dislike_query)
        does_not_dislike_dessert = not dislikes_dessert

        all_conditions_met = not_assigned_any_dessert and does_not_dislike_dessert

        # Add to KB if all conditions are met
        if all_conditions_met:
            definite_clauses_KB.tell(expr(f"Desert_{person}_{dessert}"))

        # Output result
        print(f"{person} likes the {dessert}: {all_conditions_met}")


Peter likes the NapoleonCake: False
Peter likes the Marmalade: True
Peter likes the Waffles: True
Betty likes the NapoleonCake: False
Betty likes the Marmalade: False
Betty likes the Waffles: False
Daddy likes the NapoleonCake: True
Daddy likes the Marmalade: True
Daddy likes the Waffles: True
Mummy likes the NapoleonCake: False
Mummy likes the Marmalade: False
Mummy likes the Waffles: False


In [55]:
for dessert in desserts:
    dessert_liked_by_anyone = False

    for person in persons:
        # Check if the person likes the dessert
        likes_dessert_query = expr(f"Desert_{person}_{dessert}")
        likes_dessert = pl_fc_entails(definite_clauses_KB, likes_dessert_query)

        if likes_dessert:
            dessert_liked_by_anyone = True
            break

    # If no one likes the dessert, print it and process it for the person who dislikes all desserts
    if not dessert_liked_by_anyone:
        print(f"No one likes {dessert}.")

        # Find a person who dislikes all desserts
        for person in persons:
            all_dislike = True
            for other_dessert in desserts:
                # Check if the person likes the dessert
                likes_dessert_query = expr(f"Desert_{person}_{other_dessert}")
                likes_dessert = pl_fc_entails(definite_clauses_KB, likes_dessert_query)

                # If the person likes any dessert, set all_dislike to False
                if likes_dessert:
                    all_dislike = False
                    break


            if all_dislike:
                print(f"{person} dislikes all desserts. Assigning {dessert} to {person}.")
                definite_clauses_KB.tell(expr(f"Desert_{person}_{dessert}"))
                break


No one likes IceCream.
AuntPolly dislikes all desserts. Assigning IceCream to AuntPolly.


In [56]:
# Check if anyone likes the Napoleon Cake
for person in persons:
    likes_dessert_query = expr(f"Desert_{person}_NapoleonCake")
    likes_dessert = pl_fc_entails(definite_clauses_KB, likes_dessert_query)
    print(f"{person} likes the Napoleon Cake: {likes_dessert}")


AuntPolly likes the Napoleon Cake: False
Peter likes the Napoleon Cake: False
Betty likes the Napoleon Cake: False
Daddy likes the Napoleon Cake: True
Mummy likes the Napoleon Cake: False


In [57]:
# Question: Which person dreams of visiting Paris?
for person in persons:
    query = expr(f'Dream_{person}_TripParis')

    result = pl_fc_entails(definite_clauses_KB, query)

    print(f"{person} dreams of Paris: {result}")


AuntPolly dreams of Paris: True
Peter dreams of Paris: False
Betty dreams of Paris: False
Daddy dreams of Paris: False
Mummy dreams of Paris: False
